# Feature Extraction

A quick notebook to test and determine the fastest ways to extract features from the data.

In [6]:
from typing import *
from messages import *
from helpers import *
from collections import Counter
import os
from datetime import datetime

In [7]:
conversations: list[Conversation] = []

for user in os.listdir(FILE_PATH):
    conversation = Conversation(os.path.join(FILE_PATH, user))

    conversations.append(conversation.messagesBetweenTime(datetime(2024, 1, 1).timestamp() * 1000, datetime(2025, 1, 1).timestamp() * 1000 - 1))



In [8]:
direct = [conversation for conversation in conversations if len(conversation.participants) == 2 and conversation.title in conversation.participants]

In [9]:
mainUser = predictUser(direct)

## Defining Categories

* Total Messages
* Total Messages Sent
* Messages Sent (top 5 people)
* Total Messages Received
* Messages Received (top 5 people)

#### Note: Stats should be limited to DMs only.

In [10]:
totalMessages = 0
totalMessagesSent = 0
totalMessagesReceived = 0

for conv in direct:
    totalMessages += len(conv.messages)
    totalMessagesSent += len(conv.messagesFrom(mainUser).messages)
    totalMessagesReceived += len(conv.messages) - len(conv.messagesFrom(mainUser).messages)

totalMessages, totalMessagesSent, totalMessagesReceived

(17371, 7477, 9894)